# Extraindo dados de uma API
- Nesse projeto usarei uma API pública de stock market: https://www.alphavantage.co/
- Para o desenvolvimento no notebook usarei variáveis de ambiente para salvar as chaves de acesso necessárias para pegar os dados via API.
- Ao lidar com uma API é sempre necessário ler a documentação dela para saber os parâmetros para puxar os dados que eu quero. https://www.alphavantage.co/documentation/

## Parâmetros da API: 
- function=TIME_SERIES_INTRADAY 
    - Parâmetro obrigatório que indica a série temporal
    - Escolhi INTRADAY pois de acordo com a minha extração os dados serão coletados em uma frequência de minutos (menor que um dia)
- symbol=AMZN 
    - A empresa escolhida foi 'Amazon' encontrada em uma lista na própria documentação (https://www.alphavantage.co/query?function=LISTING_STATUS&apikey=demo141)
- interval=1min
    - Intervalo escolhido de 1min pois a frequência da minha coleta é de 2min, e assim garante não ter dados duplicados. 
    - Caso não existisse 1min, escolheria 5min e trataria os dados duplicados antes de seguir com o processamento.
- apikey=os.getenv('API_KEY') 
    - Definindo valor da minha apikey para não ficar exposta no código usando variavéis de ambiente escrita no arquivo .env
        - Criar um arquivo pelo notepad chamado .env , na mesma pasta do script python.
        - Dentro desse arquivo estarão as variáveis de ambiente, como por exemplo a chave fictícia abaixo:
            - API_KEY=ERHT41AOE7F9RXXX
    - A chave foi solicitada em: https://www.alphavantage.co/support/#api-key
- Os demais parâmetros opcionais foram mantidos com os valores padrões listados na documentação.

## Códigos de Status HTTP Comuns

Os códigos de status HTTP são padrões de resposta emitidos por um servidor na Internet para indicar o resultado de uma solicitação (**request**). Aqui estão alguns dos mais comuns:

-  200 OK
    - **Descrição:** A solicitação foi bem-sucedida. O significado do sucesso varia dependendo do método HTTP.

-  400 Bad Request
    - **Descrição:** O servidor não conseguiu interpretar a solicitação devido à sintaxe inválida.

-  401 Unauthorized
    - **Descrição:** Autenticação é necessária e falhou ou ainda não foi fornecida.

-  403 Forbidden
    - **Descrição:** O cliente não tem direitos de acesso ao conteúdo; ou seja, está não autorizado, então o servidor está recusando dar a resposta.

-  404 Not Found
    - **Descrição:** O servidor não conseguiu encontrar o recurso solicitado.

-  500 Internal Server Error
    - **Descrição:** O servidor encontrou uma situação com a qual não sabe lidar.

-  503 Service Unavailable
    - **Descrição:** O servidor não está pronto para lidar com a solicitação devido a uma sobrecarga temporária ou manutenção.


> No caso, queremos que o status seja 200, para seguir com o código

## Endpoints

"Endpoint" refere-se a um ponto específico em uma API (Application Programming Interface) da web ao qual as requisições são enviadas. Os endpoints são essencialmente URLs específicas que representam diferentes funcionalidades ou recursos de uma API. Eles atuam como pontos de acesso para os dados ou serviços que a API oferece.

No meu código, o endpoint é definido pela variável url:
```python
url = f'https://www.alphavantage.co/query?function={FUNCTION}&symbol={SYMBOL}&interval={INTERVAL}&apikey={API_KEY}'
```

#### Decomposiçao do Endpoint
- Base URL: https://www.alphavantage.co/query

Esta é a base da URL que direciona a requisição para o servidor da Alpha Vantage.

- Query Parameters:

    - function={FUNCTION}: Especifica o tipo de função da API que quero acessar, neste caso, 'TIME_SERIES_INTRADAY'.
    - symbol={SYMBOL}: Define o símbolo do ativo financeiro para o qual os dados são solicitados, como 'AMZN' para a Amazon.
    - interval={INTERVAL}: Indica o intervalo de tempo
dos dados, neste caso, '1min'.

    - apikey={API_KEY}: É a chave de autenticação fornecida pela Alpha Vantage para usar a API.

#### Request-response
Cada vez que o código faz uma requisição (**request**) GET para esse endpoint:
```python
response = requests.get(url)
```
ele está solicitando dados específicos da Alpha Vantage API, de acordo com os parâmetros definidos. O servidor da API, ao receber essa requisição, processa-a e retorna uma resposta (**response**) (dados financeiros intradiários para o ativo especificado, neste caso).

> Portanto, um endpoint é um ponto de interação entre o código (cliente) e o servidor da API, permitindo que eu solicite e receba dados específicos.


# Script de Extração de Dados da API Alpha Vantage

Este script em Python realiza a extração de dados financeiros da API Alpha Vantage a cada 2 minutos durante um período de 10 minutos e salva as respostas em arquivos JSON.

## Funcionalidades Principais

- **Importação de Módulos**: Usa `requests` para requisições à API, `time` para intervalos entre requisições, `os` e `dotenv` para carregar variáveis de ambiente, `datetime` para manipulação de tempo, e `json` para salvar dados.

- **Configuração e Variáveis**: Carrega a chave da API (`API_KEY`) e outros parâmetros (como `FUNCTION`, `SYMBOL` e `INTERVAL`) de um arquivo `.env`.

- **Construção da URL da API**: Monta a URL para acessar a API Alpha Vantage com os parâmetros definidos.

- **Execução e Salvamento de Dados**: Executa um loop por 10 minutos, fazendo requisições a cada 2 minutos. As respostas são processadas em JSON e salvas em arquivos dentro de uma pasta criada com base na data e hora da execução do script.

- **Tratamento de Erros**: Utiliza `try-except` para lidar com possíveis erros durante as requisições à API.

> O script é ideal para coletar dados financeiros em tempo real e pode ser adaptado para diferentes ativos e intervalos de tempo conforme necessário.


In [5]:
## Instalação dos pacotes necessários
# %pip install requests python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests #pacote usado para fazer uma request (pegar os dados) da API
import time # para marcar o intervalo entre requests
import os
from dotenv import load_dotenv # para pegar as variaveis de ambiente 
from datetime import datetime, timedelta
import json


# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Definindo os parâmetros da API
API_KEY = os.getenv('API_KEY')
FUNCTION = 'TIME_SERIES_INTRADAY'
SYMBOL = 'AMZN'
INTERVAL = '1min'

# URL da API Alpha Vantage com os parâmetros definidos
url = f'https://www.alphavantage.co/query?function={FUNCTION}&symbol={SYMBOL}&interval={INTERVAL}&apikey={API_KEY}'

# Nome da pasta baseado na data e hora de execução do script
folder_name = f"api-extract-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
os.makedirs(folder_name, exist_ok=True)

# Inicializa o tempo de término para daqui a 10 minutos : end_time = tempo atual + 10 min
end_time = datetime.now() + timedelta(minutes=10)

while datetime.now() < end_time: # enquanto hora atual < end_time calculado
    try:
        # Faz a chamada para a API para pegar os dados (request)
        response = requests.get(url)
        response.raise_for_status()  # Levantará um erro se o status não for 200

        # Processa a resposta em JSON
        print(response.json())

        # Gera um nome de arquivo com o timestamp atual
        file_name = f"{folder_name}/{datetime.now().strftime('%Y%m%d-%H%M')}-api-response.json"

        # Salva a resposta em um arquivo JSON
        with open(file_name, 'w') as file:
            json.dump(response.json(), file)

        print(f"Resposta salva em: {file_name}")

    except requests.HTTPError as e:
        # Erro específico para códigos de status HTTP ruins (ex: 404, 500)
        print(f"Erro ao acessar a API: Status Code {response.status_code}")

    except requests.RequestException as e:
        # Outros erros de requisição (ex: problemas de rede)
        print(f"Ocorreu um erro ao fazer a chamada à API: {e}")

    # Espera por 2 minutos antes da próxima iteração (request)
    time.sleep(120)


{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Resposta salva em: api-extract-20240114-123614/20240114-1236-api-response.json
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Resposta salva em: api-extract-20240114-123614/20240114-1238-api-response.json
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
Resposta salva em: api-extract-20240114-123614/20240114-1240-api-response.json
{'Information': 'Thank you for using Alpha Vantage! Our stand

# Script para função Lambda na AWS
Nota : 'event' e 'context' nao são utilizados, entao foi colocado '_' no início

In [22]:
#########
# Script da Lambda
#########

import json
import boto3
import requests
import time
from datetime import datetime, timedelta
import os

def lambda_handler(_event, _context):
    """
    Realiza a coleta de dados de uma API financeira (Alpha Vantage) a cada 2 minutos, por um total de 10 minutos,
    e salva os resultados em arquivos JSON no Amazon S3.

    As variáveis de ambiente são utilizadas para definir os parâmetros da API e o nome do bucket do S3.
    A função registra cada ação no CloudWatch para monitoramento.

    Args:
        event: Objeto de evento que disparou a função Lambda (não utilizado neste script).
        context: Objeto de contexto fornecido pela AWS Lambda (não utilizado neste script).

    Raises:
        HTTPError: Erro se a API retornar um código de status HTTP não bem-sucedido.
        RequestException: Erro para problemas gerais de requisição (rede, etc.).
    """
    
    # Carrega as variáveis de ambiente da Lambda
    API_KEY = os.getenv('API_KEY')
    FUNCTION = os.getenv('FUNCTION')
    SYMBOL = os.getenv('SYMBOL')
    INTERVAL = os.getenv('INTERVAL')
    BUCKET_NAME = os.getenv('BUCKET_NAME')

    # URL da API Alpha Vantage com os parâmetros definidos
    url = f'https://www.alphavantage.co/query?function={FUNCTION}&symbol={SYMBOL}&interval={INTERVAL}&apikey={API_KEY}'

    # Cliente S3
    s3_client = boto3.client('s3')

    # Nome da pasta baseado na data de execução do script
    folder_name = f"api-extract-{datetime.now().strftime('%Y%m%d')}"

    # Calculando o tempo de término para daqui a 10 minutos
    end_time = datetime.now() + timedelta(minutes=10)

    while datetime.now() < end_time:
        try:
            # Faz a chamada para a API para pegar os dados
            response = requests.get(url)
            response.raise_for_status()  # Levanta um erro se o status não for 200

            # Gera um nome de arquivo com o timestamp atual até registro de minuto
            file_name = f"{folder_name}/{datetime.now().strftime('%Y%m%d-%H%M')}-api-response.json"

            # Salva a resposta em um arquivo JSON no S3 - put_object
            s3_client.put_object(Body=json.dumps(response.json()), Bucket=BUCKET_NAME, Key=file_name)

            print(f"Resposta salva no S3: {file_name}")

        except requests.HTTPError as e:
            # Erro específico para códigos de status HTTP ruins
            print(f"Erro ao acessar a API: Status Code {response.status_code}")

        except requests.RequestException as e:
            # Outros erros de requisição
            print(f"Ocorreu um erro ao fazer a chamada à API: {e}")

        # Espera por 2 minutos antes da próxima iteração
        time.sleep(120)

    # Print para monitoramento no cloudwatch
    print(f"Coleta e armazenamento de dados finalizados com sucesso. Os arquivos foram salvos em {BUCKET_NAME}/{folder_name}")
